In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, IntText

# Load the data
file_path = '/Users/osbor/OneDrive/Desktop/KDDM2/train_new.npy'
data = np.load(file_path)
data = np.round(data, 0)
#move_frames = data[:, :, ::260]


def plot_frame(frame_number):
    plt.figure(figsize=(4, 4))
    plt.imshow(data[:, :, frame_number], cmap='viridis', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title(f'Frame {frame_number}')
    plt.clim(-1, 1)  # Set the color scale limits to match your data range
    plt.show()
    

# Create an interactive text box to input the frame number
frame_text = IntText(value=0, description='Frame Number:', min=0, max=data.shape[2] - 1)

# Display the textbox and plot
interact(plot_frame, frame_number=frame_text)

ModuleNotFoundError: No module named 'numpy'

In [2]:
import chess
import chess.svg
from IPython.display import display, SVG, clear_output
import numpy as np

def create_board_from_data(data_frame):
    board = chess.Board(None)  # Create an empty board
    for i in range(8):
        for j in range(8):
            value = data_frame[i, j]
            square = chess.square(j, 7 - i)  # Flip the row index for the chess library
            
            # Assign pieces based on the value in the position
            if i == 1 and value == -1:  # Black pawns
                board.set_piece_at(square, chess.Piece(chess.PAWN, chess.BLACK))
            elif i == 6 and value == 1:  # White pawns
                board.set_piece_at(square, chess.Piece(chess.PAWN, chess.WHITE))
            elif i == 0 and value == -1:  # Black non-pawn pieces
                piece_type = [chess.ROOK, chess.KNIGHT, chess.BISHOP, chess.QUEEN, 
                              chess.KING, chess.BISHOP, chess.KNIGHT, chess.ROOK][j]
                board.set_piece_at(square, chess.Piece(piece_type, chess.BLACK))
            elif i == 7 and value == 1:  # White non-pawn pieces
                piece_type = [chess.ROOK, chess.KNIGHT, chess.BISHOP, chess.QUEEN, 
                              chess.KING, chess.BISHOP, chess.KNIGHT, chess.ROOK][j]
                board.set_piece_at(square, chess.Piece(piece_type, chess.WHITE))
    
    return board

def find_from_square(current_frame, previous_frame, to_square, board):
    for i in range(8):
        for j in range(8):
            if previous_frame[i, j] != 0 and current_frame[i, j] == 0:
                from_square = chess.square(j, 7 - i)
                if chess.Move(from_square, to_square) in board.legal_moves:
                    return from_square
    return None


def update_board(current_frame, previous_frame, board):
    for i in range(8):
        for j in range(8):
            current_value = current_frame[i, j]
            previous_value = previous_frame[i, j]
            to_square = chess.square(j, 7 - i)

            if current_value != previous_value:
                if current_value != 0:  # A piece has appeared here
                    from_square = find_from_square(current_frame, previous_frame, to_square, board)
                    if from_square:
                        move = chess.Move(from_square, to_square)
                        if move in board.legal_moves:
                            # print(f"Applying move from {from_square} to {to_square}")
                            board.push(move)
    

# Load your data
file_path = '/Users/osbor/OneDrive/Desktop/KDDM2/train_new.npy'
data = np.load(file_path)
data = np.round(data, 0)

# Extract frames after every move, which occurs every 260 frames
move_frames = data[:, :, ::260]

# Specify the desired size (in pixels)
size = 300

# Initialize the board to the standard starting position
initial_board = chess.Board()
print("Starting Board")

# Display the initial board
board_svg = chess.svg.board(initial_board, size=size)
display(SVG(board_svg))


for i in range(move_frames.shape[2]):
    previous_frame = move_frames[:, :, i - 1]
    current_frame = move_frames[:, :, i]
    update_board(current_frame, previous_frame, initial_board)
    
    clear_output(wait=True)
    
    # Debug: Print the board's FEN and a text representation
    # print("FEN:", initial_board.fen())
    # print(initial_board)

    # Display the board as an SVG
    board_svg = chess.svg.board(initial_board, size=size)
    display(SVG(board_svg))
    
    # Pause for the user to check the board
    input("Press Enter to continue to the next move...")



ModuleNotFoundError: No module named 'chess'